In [1]:
from openai import OpenAI
import pandas as pd
import numpy as np
import json
import os
import re
import time
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.prompt import PromptTemplate #where we change the AI "personality"
from dotenv import load_dotenv, find_dotenv

In [2]:
load_dotenv()
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

In [3]:
_ = load_dotenv(find_dotenv()) # read local .env file

#openai.api_key = os.getenv("api.txt")
COMPLETIONS_MODEL = "text-davinci-002"
BETTER_COMPLETIONS_MODEL = "text-davinci-003" #for my purposes, this is better
LONG_MODEL = "gpt-3.5-turbo-16k"
REGULAR_MODEL = "gpt-3.5-turbo"
GPT_4 = "gpt-4-1106-preview"

In [4]:
os.chdir('/Users/chrissoria/Documents/Research/Categorization_AI_experiments')
current_directory = os.getcwd()
print(current_directory)

/Users/chrissoria/Documents/Research/Categorization_AI_experiments


In [5]:
survey_participant_input = "e1b" #enter column name here
UCNets = pd.read_excel("../UCNets_Classification/Hand_Coding_Surveys/e1b/e1ab_Master.xlsx", engine='openpyxl', sheet_name="Master_coded")

UCNets = UCNets[['Response']]
UCNets.rename(columns={'Response': 'e1b'}, inplace=True)

UCNets = UCNets[survey_participant_input].dropna().unique()  # Drop NaN values and get unique elements

survey_participant_responses = '; '.join(str(item) for item in UCNets) #what we will feed to the model

UCNets = pd.DataFrame(UCNets, columns=[survey_participant_input])
UCNets[survey_participant_input] = UCNets[survey_participant_input].astype(str).str.lower()
UCNets[survey_participant_input] = UCNets[survey_participant_input].str.strip()
UCNets = UCNets[UCNets[survey_participant_input] != ''].reset_index(drop=True) #trimming all empty rows

UCNets = UCNets.iloc[:400]

UCNets.head()

,e1b
0,"probably don't have, because not many relatives"
1,"definitely have, because they love me"
2,"might have, because they live pretty far away."
3,"definitely have, because they love me and woul..."
4,"definitely have, because they've been there fo..."


Here, I'm trying to "force" the model to "think" in steps by first A. trying to process the response into its own words and B. having it interact with that object. That is, instead of all steps being given at once, I'm having it think in steps. 

This time, I will have it think in a "chain," where I will have it output a response and then feed that response back to it in a seperate prompt. 

In [19]:
def extract_categories(survey_question, 
                       survey_input,
                       user_model,
                       creativity,
                       categories):
    
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    
    categories_str = "\n".join(f"{i + 1}. {cat}" for i, cat in enumerate(categories))
    cat_num = len(user_categories)
    category_dict = {str(i+1): 0 for i in range(cat_num)}
    example_JSON = json.dumps(category_dict, indent=4)
    
    link1 = []
    extracted_jsons = []
    
    for response in survey_input:
        prompt = f"""Categorize this survey response "{response}" into all of the following reasons why they can or cannot rely on family and select all that apply: \
        {categories_str} \
        Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values."""
        print(prompt)
        try:
            response = client.chat.completions.create(
                model=user_model,
                messages=[
                    {'role': 'user', 'content': prompt}
                ],
                temperature=creativity
            )

            reply = response.choices[0].message.content
            link1.append(reply)
            
            except Exception as e:
                print(f"An error occurred: {e}")
                link1.append(f"Error processing input: {input}")
            
            extracted_json = re.findall(r'```json\n(\{.*?\})\n```', reply, re.DOTALL)
            
            if extracted_json:
                cleaned_json = extracted_json[0].replace('[', '').replace(']', '').replace('\n', '').replace(" ", '').replace("  ", '')
                extracted_jsons.append(cleaned_json)
                print(cleaned_json)
            else:
                error_message = """{"1":"e"}"""
                extracted_jsons.append(error_message)
                print(error_message)
            
    normalized_data_list = []
    error_lines = []
    
    for i, json_str in enumerate(extracted_jsons):
        try:
            # Attempt to parse the JSON string
            parsed_obj = json.loads(json_str)
            # Convert the parsed object to a DataFrame and append
            normalized_data_list.append(pd.json_normalize(parsed_obj))
        except json.JSONDecodeError:
            # Define a default JSON object as a dictionary
            default_json_obj = {"1": "j"}
            # Convert the default object to a DataFrame and append
            normalized_data_list.append(pd.json_normalize(default_json_obj))

    normalized_data = pd.concat(normalized_data_list, ignore_index=True)
    
    categorized_data = pd.DataFrame()
    categorized_data['survey_response'] = survey_input
    categorized_data['link1'] = link1
    categorized_data['json'] = extracted_jsons
    
    categorized_data = pd.concat([categorized_data, normalized_data], axis=1)
    
    return categorized_data

SyntaxError: invalid syntax (33032466.py, line 34)

In [12]:
def extract_categories(survey_question, 
                       survey_input,
                       user_model,
                       creativity,
                       categories):
    
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    
    categories_str = "\n".join(f"{i + 1}. {cat}" for i, cat in enumerate(categories))
    cat_num = len(categories)
    category_dict = {str(i+1): 0 for i in range(cat_num)}
    example_JSON = json.dumps(category_dict, indent=4)
    
    link1 = []
    extracted_jsons = []
    
    for response in survey_input:
        prompt = f"""A survey respondent was asked, "{survey_question}" \
        Their response is here in triple backticks: ```{response}```. \
        First, thoruoughly extract all their answers to the question and be as specific as possible. \
        Second, take these reasons and select all of the following numbered categories they fall into: \
        "{categories_str}" \
        Third, form your response in proper JSON format. \
        The number belonging to the category shoulbe be the key and a 1 is the key value if the category is present. \
        If none of the categories are present in their response, provide 0's for all key values in your JSON."""
        print(prompt)
        try:
            response = client.chat.completions.create(
                model=user_model,
                messages=[
                    {
                      "role": "system",
                      "content": f"""You are an expert in identifying themes and patterns in open-ended survey responses to the question, "{survey_question}". \
                      When given a survey response, you analyze it critically and thoroughly to identify user-provided categories present in the response."""
                    },
                    {'role': 'user', 
                     'content': prompt}
                ],
                temperature=creativity
            )

            reply = response.choices[0].message.content
            link1.append(reply)
            
            extracted_json = re.findall(r'```json\n(\{.*?\})\n```', reply, re.DOTALL)
            
            if extracted_json:
                cleaned_json = extracted_json[0].replace('[', '').replace(']', '').replace('\n', '').replace(" ", '').replace("  ", '')
                extracted_jsons.append(cleaned_json)
                print(cleaned_json)
            else:
                error_message = """{"1":"e"}"""
                extracted_jsons.append(error_message)
                print(error_message)
            
        except Exception as e:
            print(f"An error occurred: {e}")
            link1.append(f"Error processing input: {input}")
            extracted_jsons.append(f"Error processing input: {input}")
            
    normalized_data_list = []
    
    for json_str in extracted_jsons:
        try:
            parsed_obj = json.loads(json_str)
            normalized_data_list.append(pd.json_normalize(parsed_obj))
        except json.JSONDecodeError:
            default_json_obj = {"1": "j"}
            normalized_data_list.append(pd.json_normalize(default_json_obj))

    normalized_data = pd.concat(normalized_data_list, ignore_index=True)
    
    categorized_data = pd.DataFrame()
    categorized_data['survey_response'] = survey_input
    categorized_data['link1'] = link1
    categorized_data['json'] = extracted_jsons
    
    categorized_data = pd.concat([categorized_data, normalized_data], axis=1)
    
    return categorized_data


In [13]:
survey_question = "If you had a serious problem, like a life-threatening illness or possibly losing your home, do you feel that you have some relatives that you can rely on to help?"
survey_input = UCNets['e1b']

creativity = .0
#category 2 and 3 are very similar and require nuance to differentiate 
user_categories = ["A simple statement without further elaboration, often expressed simply as 'because I know they would help'",
                   "Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.",
                   "Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family support, or values that relatives inherently help each other, again applying to families in general.",
                   "Their own family being particularly close, loyalty,  as well as emotional support and love within their own family (rather than references to families in general).",
                   "Mention of a family member's personal character, such as 'they are a generous person' or 'they aren't good people'.",
                   "Past instances of receiving help or support from family members",
                   "Highlighting relatives' unreliability in times of need, due to factors such as being too busy, lack of willpower, or other reasons causing unreliablity.",
                   "Referencing a pattern of mutual aid and reciprocal help within the family. A give-and-take dynamic. Here, unlike category 4, there are suggestions that the respondent has helped his or her relatives.",
                   "Estrangement, tension, hostility, or indifference within the family. Respondents may mention lack of communication, lack of trust, or unwillingness or inability (both practical and emotional) to ask for help from relatives.",
                   "Indication that the person wouldn't or prefers not to rely on family, or that they do not feel the need to. Reliance on friends (or no one) rather than family for support.",
                   "Mention their relatives' general resources as a factor, either enabling or preventing them from providing assistance.",
                   "Mention their relatives' financial resources as a factor, either enabling or preventing them from providing assistance.",
                   "Their family's geographic proximity as a factor, either enabling or hindering the provision of support during times of crisis.",
                   "They have no relatives, for whatever reason.",
                   "They were verbally told by a family member that they could rely on them.",
                   "They mention the duration of the relationship as a factor",
                   "They reference a single person",
                   "Their family can provide support during a crisis due to reasons not covered in the other categories.",
                   "Their family cannot or won't provide support during a crisis due to reasons not encompassed in the other categories.",
                   "Not clear / Not categorizable"]

user_model = 'gpt-4-1106-preview'

In [14]:
bad = extract_categories(survey_question, 
                            survey_input, 
                            user_model,
                            creativity,
                            user_categories)

bad.to_csv('data/e1b_bad_categorization_5_cats.csv')

A survey respondent was asked, "If you had a serious problem, like a life-threatening illness or possibly losing your home, do you feel that you have some relatives that you can rely on to help?"         Their response is here in triple backticks: ```probably don't have, because not many relatives```.         First, thoruoughly extract all their answers to the question and be as specific as possible.         Second, take these reasons and select all of the following numbered categories they fall into:         "1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family 

KeyboardInterrupt: 

In [15]:
categorized_data = extract_categories(survey_question, 
                            survey_input, 
                            api_key,
                            user_categories)

NameError: name 'api_key' is not defined

In [ ]:
print(len(bad))

In [ ]:
def extract_categories_improved(survey_question, 
                       survey_input,
                       user_model,
                       creativity,
                       categories):
    
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    
    categories_str = "\n".join(f"{i + 1}. {cat}" for i, cat in enumerate(categories))
    cat_num = len(user_categories)
    category_dict = {str(i+1): "0" for i in range(cat_num)}
    example_JSON = json.dumps(category_dict, indent=4)
    
    link1 = []
    extracted_jsons = []
    
    for response in survey_input:
        prompt = f"""A survey respondent was asked, "{survey_question}" \
        Their response is here in triple backticks: ```{response}```. \
        Select all of the following numbered categories present in the response and form your response in proper JSON format: \
        The number belonging to the category should be be the key and a 1 is the key value if the category is present. \
        If none of the categories are present in their response, provide 0's for all key values in your JSON. \
        Numbered categories: "{categories_str}"."""

        try:
            response = client.chat.completions.create(
                model=user_model,
                messages=[
                    {
                      "role": "system",
                      "content": f"""You are an expert in identifying themes and patterns in open-ended survey responses to the question, "{survey_question}". \
                      When given a survey response, you analyze it critically and thoroughly to identify user-provided categories present in the response."""
                    },
                    {'role': 'user', 
                     'content': prompt}
                ],
                temperature=creativity
            )

            reply = response.choices[0].message.content
            link1.append(reply)
            
        except Exception as e:
            print(f"An error occurred: {e}")
            link1.append(f"Error processing input: {input}")
            
        extracted_json = re.findall(r'```json\n(\{.*?\})\n```', reply, re.DOTALL)
            
        if extracted_json:
            cleaned_json = extracted_json[0].replace('[', '').replace(']', '').replace('\n', '').replace(" ", '').replace("  ", '')
            extracted_jsons.append(cleaned_json)
            print(cleaned_json)
        else:
            error_message = """{"1":"e"}"""
            extracted_jsons.append(error_message)
            print(error_message)
            
    normalized_data_list = []
    error_lines = []
    
    for i, json_str in enumerate(extracted_jsons):
        try:
            parsed_obj = json.loads(json_str)
            normalized_data_list.append(pd.json_normalize(parsed_obj))
        except json.JSONDecodeError:
            default_json_obj = {"1": "j"}
            normalized_data_list.append(pd.json_normalize(default_json_obj))

    normalized_data = pd.concat(normalized_data_list, ignore_index=True)
    
    categorized_data = pd.DataFrame()
    categorized_data['survey_response'] = survey_input
    categorized_data['link1'] = link1
    categorized_data['json'] = extracted_jsons
    
    categorized_data = pd.concat([categorized_data, normalized_data], axis=1)
    
    return categorized_data

In [8]:
good = extract_categories_improved(survey_question, 
                            survey_input, 
                            user_model,
                            creativity,
                            user_categories)

good.to_csv('data/e1b_good_categorization_5_cats.csv')

NameError: name 'extract_categories_improved' is not defined

In [10]:
def extract_categories_cot(survey_question, 
                       survey_input,
                       user_model,
                       creativity,
                       categories):
    
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    
    categories_str = "\n".join(f"{i + 1}. {cat}" for i, cat in enumerate(categories))
    cat_num = len(user_categories)
    category_dict = {str(i+1): "0" for i in range(cat_num)}
    example_JSON = json.dumps(category_dict, indent=4)
    
    link1 = []
    extracted_jsons = []
    
    for response in survey_input:
        prompt = f"""A survey respondent was asked, "{survey_question}" \
        Their response is here in triple backticks: ```{response}```. \
        First, thoruoughly extract all their answers to the question and be as specific as possible. \
        Second, take these reasons and select all of the following numbered categories they fall into: \
        "{categories_str}" \
        Third, form your response in proper JSON format. \
        The number belonging to the category shoulbe be the key and a 1 is the key value if the category is present. \
        If none of the categories are present in their response, provide 0's for all key values in your JSON."""
        print(prompt)
        try:
            response = client.chat.completions.create(
                model=user_model,
                messages=[
                    {
                      "role": "system",
                      "content": f"""You are an expert in identifying themes and patterns in open-ended survey responses to the question, "{survey_question}". \
                      When given a survey response, you analyze it critically and thoroughly to identify user-provided categories present in the response."""
                    },
                    {'role': 'user', 
                     'content': prompt}
                ],
                temperature=creativity
            )

            reply = response.choices[0].message.content
            link1.append(reply)
            
        except Exception as e:
            print(f"An error occurred: {e}")
            link1.append(f"Error processing input: {input}")
            
        extracted_json = re.findall(r'```json\n(\{.*?\})\n```', reply, re.DOTALL)
            
        if extracted_json:
            cleaned_json = extracted_json[0].replace('[', '').replace(']', '').replace('\n', '').replace(" ", '').replace("  ", '')
            extracted_jsons.append(cleaned_json)
            print(cleaned_json)
        else:
            error_message = """{"1":"e"}"""
            extracted_jsons.append(error_message)
            print(error_message)
            
    normalized_data_list = []
    error_lines = []
    
    for i, json_str in enumerate(extracted_jsons):
        try:
            parsed_obj = json.loads(json_str)
            normalized_data_list.append(pd.json_normalize(parsed_obj))
        except json.JSONDecodeError:
            default_json_obj = {"1": "j"}
            normalized_data_list.append(pd.json_normalize(default_json_obj))

    normalized_data = pd.concat(normalized_data_list, ignore_index=True)
    
    categorized_data = pd.DataFrame()
    categorized_data['survey_response'] = survey_input
    categorized_data['link1'] = link1
    categorized_data['json'] = extracted_jsons
    
    categorized_data = pd.concat([categorized_data, normalized_data], axis=1)
    
    return categorized_data



IndentationError: unindent does not match any outer indentation level (<tokenize>, line 27)

In [10]:
cot = extract_categories_cot(survey_question, 
                            survey_input, 
                            user_model,
                            creativity,
                            user_categories)

cot.to_csv('data/e1b_cot_categorization_20_cats.csv')

A survey respondent was asked, "If you had a serious problem, like a life-threatening illness or possibly losing your home, do you feel that you have some relatives that you can rely on to help?"         Their response is here in triple backticks: ```probably don't have, because not many relatives```.         First, thoruoughly extract all their answers to the question and be as specific as possible.         Second, take these reasons and select all of the following numbered categories they fall into:         "1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family 

UnboundLocalError: local variable 'reply' referenced before assignment

In [ ]:
def extract_categories_cove(survey_question, 
                       survey_input,
                       user_model,
                       creativity,
                       categories):
    
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    
    categories_str = "\n".join(f"{i + 1}. {cat}" for i, cat in enumerate(categories))
    cat_num = len(categories)
    category_dict = {str(i+1): "0" for i in range(cat_num)}
    example_JSON = json.dumps(category_dict, indent=4)
    
    link1 = []
    link2 = []
    extracted_jsons = []
    
    for response in survey_input:
        prompt = f"""Categorize this survey response "{response}" into all of the following reasons why they can or cannot rely on family and select all that apply: \
        {categories_str} \
        Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values."""
        try:
            api_response = client.chat.completions.create(
                model=user_model,
                messages=[
                    {'role': 'user', 
                     'content': prompt}
                ],
                temperature=creativity
            )
            reply = api_response.choices[0].message.content
            print(reply)
            link1.append(reply)

            prompt2 = f"""Thank you for categorizing this survey response, "{response}". \
            Can you double check if there are any categories you might've missed or marked as being present incorrectly? \
            Here are the categories once again: {categories_str} \
            If there are any changes, please output a corrected JSON with the new categorization. \
            If there are no changes, please output the original JSON."""
            print(prompt2)
            

            api_response2 = client.chat.completions.create(
                model=user_model,
                messages=[
                    {"role": "system", "content": f"""You just categorized an answer to the question, "{survey_question}". You are revisiting your initial categorizations for accuracy. Here's what you initially identified: {reply}"""}, 
                    {'role':'assistant', 'content': reply},
                    {'role': 'user', 'content': prompt2}
                ],
                temperature=.25,
            )

            reply2 = api_response2.choices[0].message.content
            link2.append(reply2)
            
            extracted_json = re.findall(r'```json\n(\{.*?\})\n```', reply, re.DOTALL)
            
            if extracted_json:
                cleaned_json = extracted_json[0].replace('[', '').replace(']', '').replace('\n', '').replace(" ", '').replace("  ", '')
                extracted_jsons.append(cleaned_json)
                print(cleaned_json)
            else:
                error_message = """{"1":"e"}"""
                extracted_jsons.append(error_message)
                print(error_message)
                
        except Exception as e:
            print(f"An error occurred: {e}")
            link1.append(f"Error processing input: {survey_input}")
            link2.append(f"Error processing response: {reply}")
            
    normalized_data_list = []
    error_lines = []
    
    for i, json_str in enumerate(extracted_jsons):
        try:
            parsed_obj = json.loads(json_str)
            normalized_data_list.append(pd.json_normalize(parsed_obj))
        except json.JSONDecodeError:
            default_json_obj = {"1": "j"}
            normalized_data_list.append(pd.json_normalize(default_json_obj))

    normalized_data = pd.concat(normalized_data_list, ignore_index=True)
    
    categorized_data = pd.DataFrame()
    categorized_data['survey_response'] = survey_input
    categorized_data['link1'] = link1
    categorized_data['link2'] = link2
    categorized_data['json'] = extracted_jsons
    
    categorized_data = pd.concat([categorized_data, normalized_data], axis=1)
    
    return categorized_data

In [ ]:
creativity = .0

cove = extract_categories_cove(survey_question, 
                            survey_input, 
                            user_model,
                            creativity,
                            user_categories)

cove.to_csv('data/e1b_cove_categorization_5_cats.csv')

In [ ]:
def extract_categories_1s(survey_question, 
                       survey_input,
                       user_model,
                       creativity,
                       categories):
    
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    
    categories_str = "\n".join(f"{i + 1}. {cat}" for i, cat in enumerate(categories))
    cat_num = len(user_categories)
    category_dict = {str(i+1): 0 for i in range(cat_num)}
    example_categorization = """{"1":0,"2":0,"3":1,"4":1,"5":0,"6":0,"7":0,"8":0,"9":0,"10":0,"11":0,"12":0,"13":0,"14":0,"15":0,"16":0,"17":0,"18":0,"19":0,"20":0}"""
    #1. going out with coworkers, 2. board games (cultural-hobby), 3. activities in the city (broader participation)
    example_response = "Definitely have, because because of our close relationships and probably the family obligation or responsiblilty."
    link1 = []
    extracted_jsons = []
    
    for response in survey_input:
        prompt = f"""Categorize this survey response "{response}" into all of the following reasons why they can or cannot rely on family and select all that apply: \
        {categories_str} \
        Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values. \
        Here's an example of a correct categorization. \
        Example survey response: {example_response}. \
        Example categorization: {example_categorization}."""
        try:
            response = client.chat.completions.create(
                model=user_model,
                messages=[
                    {'role': 'user', 'content': prompt}
                ],
                temperature=creativity
            )

            reply = response.choices[0].message.content
            link1.append(reply)
            
        except Exception as e:
            print(f"An error occurred: {e}")
            link1.append(f"Error processing input: {input}")
           
        extracted_json = re.findall(r'```json\n(\{.*?\})\n```', reply, re.DOTALL)
            
        if extracted_json:
            cleaned_json = extracted_json[0].replace('[', '').replace(']', '').replace('\n', '').replace(" ", '').replace("  ", '')
            extracted_jsons.append(cleaned_json)
            print(cleaned_json)
        else:
            error_message = """{"1":"e"}"""
            extracted_jsons.append(error_message)
            print(error_message)
            
    normalized_data_list = []
    error_lines = []
    
    for i, json_str in enumerate(extracted_jsons):
        try:
            parsed_obj = json.loads(json_str)
            normalized_data_list.append(pd.json_normalize(parsed_obj))
        except json.JSONDecodeError:
            default_json_obj = {"1": "j"}
            normalized_data_list.append(pd.json_normalize(default_json_obj))

    normalized_data = pd.concat(normalized_data_list, ignore_index=True)
    
    categorized_data = pd.DataFrame()
    categorized_data['survey_response'] = survey_input
    categorized_data['link1'] = link1
    categorized_data['json'] = extracted_jsons
    
    categorized_data = pd.concat([categorized_data, normalized_data], axis=1)
    
    return categorized_data

In [ ]:
oneshot = extract_categories_1s(survey_question, 
                            survey_input, 
                            user_model,
                            creativity,
                            user_categories)

oneshot.to_csv('data/e1b_1s_categorization_5_cats.csv')

In [ ]:
def extract_categories_fs(survey_question, 
                       survey_input,
                       user_model,
                       creativity,
                       categories):
    
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    
    categories_str = "\n".join(f"{i + 1}. {cat}" for i, cat in enumerate(categories))
    cat_num = len(user_categories)
    category_dict = {str(i+1): "0" for i in range(cat_num)}
    example_response = "Definitely have, because because of our close relationships and probably the family obligation or responsiblilty."
    example_categorization = """{"1":0,"2":0,"3":1,"4":1,"5":0,"6":0,"7":0,"8":0,"9":0,"10":0,"11":0,"12":0,"13":0,"14":0,"15":0,"16":0,"17":0,"18":0,"19":0,"20":0}"""
    example_response2 = "school kind of provides a network, so that's basically all. i also joined some political organizations outside of school"
    example_categorization2 = """{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0,"7":0,"8":0,"9":1}"""
    example_response3 = "going to local breweries, wineries and bar as well as joining a recreational league."
    example_categorization3 = """{"1":0,"2":1,"3":0,"4":0,"5":0,"6":1,"7":0,"8":0,"9":0}"""
    
    link1 = []
    extracted_jsons = []
    
    for response in survey_input:
        prompt = f"""Categorize this survey response "{response}" into all of the following reasons for moving and select all that apply: \
        {categories_str} \
        Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values. \
        Here are three examples of a correct categorization. \
        Example survey response 1: {example_response}. \
        Example categorization 1: {example_categorization}. \
        Example survey response 2: {example_response2}. \
        Example categorization 2: {example_categorization2}. \
        Example survey response 3: {example_response3}. \
        Example categorization 3: {example_categorization3}."""
        try:
            response = client.chat.completions.create(
                model=user_model,
                messages=[
                    {'role': 'user', 'content': prompt}
                ],
                temperature=creativity
            )

            reply = response.choices[0].message.content
            link1.append(reply)
            
        except Exception as e:
            print(f"An error occurred: {e}")
            link1.append(f"Error processing input: {input}")
            
        extracted_json = re.findall(r'```json\n(\{.*?\})\n```', reply, re.DOTALL)
            
        if extracted_json:
            cleaned_json = extracted_json[0].replace('[', '').replace(']', '').replace('\n', '').replace(" ", '').replace("  ", '')
            extracted_jsons.append(cleaned_json)
            print(cleaned_json)
        else:
            error_message = """{"1":"e"}"""
            extracted_jsons.append(error_message)
            print(error_message)
            
    normalized_data_list = []
    error_lines = []
    
    for i, json_str in enumerate(extracted_jsons):
        try:
            parsed_obj = json.loads(json_str)
            normalized_data_list.append(pd.json_normalize(parsed_obj))
        except json.JSONDecodeError:
            default_json_obj = {"1": "j"}
            normalized_data_list.append(pd.json_normalize(default_json_obj))

    normalized_data = pd.concat(normalized_data_list, ignore_index=True)
    
    categorized_data = pd.DataFrame()
    categorized_data['survey_response'] = survey_input
    categorized_data['link1'] = link1
    categorized_data['json'] = extracted_jsons
    
    categorized_data = pd.concat([categorized_data, normalized_data], axis=1)
    
    return categorized_data

In [ ]:
fewshot = extract_categories_fs(survey_question, 
                            survey_input, 
                            user_model,
                            creativity,
                            user_categories)

fewshot.to_csv('data/a19f_fs_categorization_5_cats.csv')

In [41]:
def extract_categories_cot_1s(survey_question, 
                       survey_input,
                       user_model,
                       creativity,
                       categories):
    
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    
    categories_str = "\n".join(f"{i + 1}. {cat}" for i, cat in enumerate(categories))
    cat_num = len(user_categories)
    category_dict = {str(i+1): "0" for i in range(cat_num)}
    example_JSON = json.dumps(category_dict, indent=4)
    example_categorization = """{"1":0,"2":0,"3":1,"4":1,"5":0,"6":0,"7":0,"8":0,"9":0,"10":0,"11":0,"12":0,"13":0,"14":0,"15":0,"16":0,"17":0,"18":0,"19":0,"20":0}"""
    #1. going out with coworkers, 2. board games (cultural-hobby), 3. activities in the city (broader participation)
    example_response = "Definitely have, because because of our close relationships and probably the family obligation or responsiblilty."
    link1 = []
    extracted_jsons = []
    
    for response in survey_input:
        prompt = f"""A survey respondent was asked, "{survey_question}" \
        Their response is here in triple backticks: ```{response}```. \
        First, thoruoughly extract all their answers to the question and be as specific as possible. \
        Second, take these reasons and select all of the following numbered categories they fall into: \
        "{categories_str}" \
        Here's an example of a correct categorization. \
        Example survey response: {example_response} \
        Example categorization: {example_categorization}. \
        If none of the categories are present in their response, provide 0's for all key values in your JSON."""
        print(prompt)
        try:
            response = client.chat.completions.create(
                model=user_model,
                messages=[
                    {
                      "role": "system",
                      "content": f"""You are an expert in identifying themes and patterns in open-ended survey responses to the question, "{survey_question}". \
                      When given a survey response, you analyze it critically and thoroughly to identify user-provided categories present in the response."""
                    },
                    {'role': 'user', 
                     'content': prompt}
                ],
                temperature=creativity
            )

            reply = response.choices[0].message.content
            link1.append(reply)
            
        except Exception as e:
            print(f"An error occurred: {e}")
            link1.append(f"Error processing input: {input}")
            
        extracted_json = re.findall(r'```json\n(\{.*?\})\n```', reply, re.DOTALL)
            
        if extracted_json:
            cleaned_json = extracted_json[0].replace('[', '').replace(']', '').replace('\n', '').replace(" ", '').replace("  ", '')
            extracted_jsons.append(cleaned_json)
            print(cleaned_json)
        else:
            error_message = """{"1":"e"}"""
            extracted_jsons.append(error_message)
            print(error_message)
            
    normalized_data_list = []
    error_lines = []
    
    for i, json_str in enumerate(extracted_jsons):
        try:
            parsed_obj = json.loads(json_str)
            normalized_data_list.append(pd.json_normalize(parsed_obj))
        except json.JSONDecodeError:
            default_json_obj = {"1": "j"}
            normalized_data_list.append(pd.json_normalize(default_json_obj))

    normalized_data = pd.concat(normalized_data_list, ignore_index=True)
    
    categorized_data = pd.DataFrame()
    categorized_data['survey_response'] = survey_input
    categorized_data['link1'] = link1
    categorized_data['json'] = extracted_jsons
    
    categorized_data = pd.concat([categorized_data, normalized_data], axis=1)
    
    return categorized_data

In [43]:
cot_1s = extract_categories_cot_1s(survey_question, 
                            survey_input, 
                            user_model,
                            creativity,
                            user_categories)

cot_1s.to_csv('data/e1b_cot1s_categorization_5_cats.csv')

A survey respondent was asked, "If you had a serious problem, like a life-threatening illness or possibly losing your home, do you feel that you have some relatives that you can rely on to help?"         Their response is here in triple backticks: ```probably don't have, because not many relatives```.         First, thoruoughly extract all their answers to the question and be as specific as possible.         Second, take these reasons and select all of the following numbered categories they fall into:         "1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional expectations of family 

{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0,"7":0,"8":0,"9":0,"10":0,"11":0,"12":0,"13":1,"14":0,"15":0,"16":0,"17":0,"18":0,"19":0,"20":0}
A survey respondent was asked, "If you had a serious problem, like a life-threatening illness or possibly losing your home, do you feel that you have some relatives that you can rely on to help?"         Their response is here in triple backticks: ```definitely have, because they love me and would do what they can to help```.         First, thoruoughly extract all their answers to the question and be as specific as possible.         Second, take these reasons and select all of the following numbered categories they fall into:         "1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are re

{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0,"7":1,"8":0,"9":1,"10":0,"11":0,"12":0,"13":0,"14":0,"15":0,"16":0,"17":0,"18":0,"19":0,"20":0}
A survey respondent was asked, "If you had a serious problem, like a life-threatening illness or possibly losing your home, do you feel that you have some relatives that you can rely on to help?"         Their response is here in triple backticks: ```definitely have, because we take care of each other, family is important. they've also done well, able to take care of each other.```.         First, thoruoughly extract all their answers to the question and be as specific as possible.         Second, take these reasons and select all of the following numbered categories they fall into:         "1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple t

{"1":0,"2":0,"3":0,"4":0,"5":1,"6":0,"7":1,"8":0,"9":1,"10":0,"11":0,"12":0,"13":0,"14":0,"15":0,"16":0,"17":1,"18":0,"19":0,"20":0}
A survey respondent was asked, "If you had a serious problem, like a life-threatening illness or possibly losing your home, do you feel that you have some relatives that you can rely on to help?"         Their response is here in triple backticks: ```definitely have, because she feels like they will always take care of her.```.         First, thoruoughly extract all their answers to the question and be as specific as possible.         Second, take these reasons and select all of the following numbered categories they fall into:         "1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are 

{"1":1,"2":1,"3":1,"4":1,"5":0,"6":0,"7":0,"8":0,"9":0,"10":0,"11":0,"12":0,"13":0,"14":0,"15":0,"16":0,"17":0,"18":0,"19":0,"20":0}
A survey respondent was asked, "If you had a serious problem, like a life-threatening illness or possibly losing your home, do you feel that you have some relatives that you can rely on to help?"         Their response is here in triple backticks: ```definitely have, because my sister lives close and has always been my favorite. she was there for my heart surgery before and we've always got along well. i have a good connection with her.```.         First, thoruoughly extract all their answers to the question and be as specific as possible.         Second, take these reasons and select all of the following numbered categories they fall into:         "1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each o

...

{"1":0,"2":0,"3":0,"4":0,"5":1,"6":0,"7":1,"8":0,"9":0,"10":0,"11":0,"12":0,"13":0,"14":0,"15":0,"16":0,"17":1,"18":0,"19":1,"20":0}
A survey respondent was asked, "If you had a serious problem, like a life-threatening illness or possibly losing your home, do you feel that you have some relatives that you can rely on to help?"         Their response is here in triple backticks: ```probably have, because yes, my sister or my husband's family```.         First, thoruoughly extract all their answers to the question and be as specific as possible.         Second, take these reasons and select all of the following numbered categories they fall into:         "1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' ap

{"1":1,"2":1,"3":0,"4":0,"5":0,"6":0,"7":0,"8":0,"9":0,"10":0,"11":0,"12":0,"13":0,"14":0,"15":0,"16":0,"17":0,"18":0,"19":0,"20":0}
A survey respondent was asked, "If you had a serious problem, like a life-threatening illness or possibly losing your home, do you feel that you have some relatives that you can rely on to help?"         Their response is here in triple backticks: ```definitely have, because i guess its the assumption that it's family. sometimes you don't know about these things```.         First, thoruoughly extract all their answers to the question and be as specific as possible.         Second, take these reasons and select all of the following numbered categories they fall into:         "1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'becaus

{"1":"e"}
A survey respondent was asked, "If you had a serious problem, like a life-threatening illness or possibly losing your home, do you feel that you have some relatives that you can rely on to help?"         Their response is here in triple backticks: ```definitely have, because they have helped me in the past.```.         First, thoruoughly extract all their answers to the question and be as specific as possible.         Second, take these reasons and select all of the following numbered categories they fall into:         "1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'because we are relatives,' applying to families in general.
3. Explicitly referring to the principle of family obligations, the cultural or traditional exp

{"1":"e"}
A survey respondent was asked, "If you had a serious problem, like a life-threatening illness or possibly losing your home, do you feel that you have some relatives that you can rely on to help?"         Their response is here in triple backticks: ```definitely have, because my brother and sister made a pact that we go through this together and we haven't had any riffs. i know we are very fortunate, if it were something crucial i would probably leave here and go stay with my sister. she would want me to.```.         First, thoruoughly extract all their answers to the question and be as specific as possible.         Second, take these reasons and select all of the following numbered categories they fall into:         "1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very sim

{"1":0,"2":0,"3":0,"4":0,"5":0,"6":0,"7":0,"8":0,"9":1,"10":0,"11":0,"12":0,"13":0,"14":0,"15":0,"16":0,"17":0,"18":0,"19":0,"20":0}
A survey respondent was asked, "If you had a serious problem, like a life-threatening illness or possibly losing your home, do you feel that you have some relatives that you can rely on to help?"         Their response is here in triple backticks: ```definitely have, because the fact that i'm doing it right now (living with relative)```.         First, thoruoughly extract all their answers to the question and be as specific as possible.         Second, take these reasons and select all of the following numbered categories they fall into:         "1. A simple statement without further elaboration, often expressed simply as 'because I know they would help'
2. Briefly indicating an implicit duty or obligation that all family members have to support each other in times of crisis, usually expressed in very simple terms like 'because they're family,' and 'becau